In [1]:
import pandas as pd
df = pd.read_csv("data.csv")
df = df.dropna()
df.shape[0]

3988

In [2]:
fake = df[df.Label ==  0]
real = df[df.Label ==  1]

In [3]:
print(fake.shape)
print(real.shape)
fake.iloc[1]

(2120, 4)
(1868, 4)


URLs        https://www.activistpost.com/2017/09/u-s-presi...
Headline    U.S. President Donald Trump Quietly Signs Law ...
Body        By Aaron Kesel\nAs former White House chief of...
Label                                                       0
Name: 10, dtype: object

In [4]:
from sklearn.model_selection import train_test_split
y = df.pop('Label')
x = df

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42, stratify = y)


In [5]:
appos = {
    "aren't" : "are not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "didn't" : "did not",
    "doesn't" : "does not",
    "don't" : "do not",
    "hadn't" : "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "he'd" : "he would",
    "he'll" : "he will",
    "he's" : "he is",
    "i'd" : "I would",
    "i'd" : "I had",
    "i'll" : "I will",
    "i'm" : "I am",
    "isn't" : "is not",
    "is'nt" : "is not",
    "it's" : "it is",
    "it'll":"it will",
    "i've" : "I have",
    "let's" : "let us",
    "mightn't" : "might not",
    "mustn't" : "must not",
    "shan't" : "shall not",
    "she'd" : "she would",
    "she'll" : "she will",
    "she's" : "she is",
    "shouldn't" : "should not",
    "that's" : "that is",
    "there's" : "there is",
    "they'd" : "they would",
    "they'll" : "they will",
    "they're" : "they are",
    "they've" : "they have",
    "we'd" : "we would",
    "we're" : "we are",
    "weren't" : "were not",
    "we've" : "we have",
    "what'll" : "what will",
    "what're" : "what are",
    "what's" : "what is",
    "what've" : "what have",
    "where's" : "where is",
    "who'd" : "who would",
    "who'll" : "who will",
    "who're" : "who are",
    "who's" : "who is",
    "who've" : "who have",
    "won't" : "will not",
    "wouldn't" : "would not",
    "you'd" : "you would",
    "you'll" : "you will",
    "you're" : "you are",
    "you've" : "you have",
    "'re": " are",
    "wasn't": "was not",
    "we'll":" will",
    "didn't": "did not"
}

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
# nltk.download('punkt')
# nltk.download('stopwords')

ps = PorterStemmer()

import string

In [7]:
#Source - https://medium.com/@annabiancajones/sentiment-analysis-of-reviews-text-pre-processing-6359343784fb
def negationHandling(word):
    if word in appos:
        return appos[word]
    else:
        return word

In [8]:
def isNotStopWord(word):
    return word not in stopwords.words('english')

In [9]:
def tokenizeDocument(document):
    sentences = nltk.sent_tokenize(document)
    tokens = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        
        #Converting to LowerCase
        words = map(str.lower, words)
        
        # Negation Handling map is'nt to is not : 
        words = map(lambda x: negationHandling(x), words)
        
        # Remove stop words
        words = filter(lambda x: isNotStopWord(x), words)
        
        # Removing punctuations except '<.>/<?>/<!>'
        punctuations = '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'
        words = map(lambda x: x.translate(str.maketrans('', '', punctuations)), words)
        
        # Remove empty strings
        words = filter(lambda x: len(x) > 0, words)
        
        # stemming
        words = map(lambda x: ps.stem(x), words)
        
        # Adding the preprocessed words to the document
        tokens = tokens + list(words)
        
    return tokens
        

In [10]:
#Pre-processing
#Training Data
#Body Column

train_Body = x_train.loc[:,'Body']
train_docBodyWordArray = []

for i in range(x_train.shape[0]):
    train_docBodyWordArray.append(train_Body.iloc[i])
        
train_BodywordArray = list(map(lambda x: tokenizeDocument(x), train_docBodyWordArray))
print("Preprocessing Completed for Body of training data")


Preprocessing Completed for Body of training data


In [11]:
#Training Data
#Headline Column
train_headLine = x_train.loc[:,'Headline']

train_docHeadLineWordArray = []
for i in range(x_train.shape[0]):
    train_docHeadLineWordArray.append(train_headLine.iloc[i])
        
train_HeadLineArray = list(map(lambda x: tokenizeDocument(x), train_docHeadLineWordArray))
print("Preprocessing Completed for HeadLine of training data")

Preprocessing Completed for HeadLine of training data


In [12]:
#conda install -c anaconda gensim 
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\Aayush\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
#Max Count of words in Document - Split by 10,000
max = 0
for i in range(df.shape[0]):
    length = len(df.iloc[i,2])
    if(length>max):
        max = length
    
max

In [13]:
tagged_DocBodyData = [TaggedDocument(words=train_BodywordArray[i], tags=[str(i)]) for i, _d in enumerate(train_BodywordArray)]

In [ ]:
#tagged_data[0]

In [14]:
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.025,
                min_count=5,
                window = 10,
                dm =1)

model.build_vocab(tagged_DocBodyData)


In [15]:

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_DocBodyData,total_examples=model.corpus_count,epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0


C:\Users\Aayush\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [16]:
model= Doc2Vec.load("d2v.model")

#test_data = word_tokenize("I love chatbots".lower())
#v1 = model.infer_vector(test_data)
#print("V1_infer", v1)

# to find most similar doc using tags
#similar_doc = model.docvecs.most_similar('1318')
#print(similar_doc)

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
#print(model.docvecs['0'])

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
#URL
#Get List of authentic news sources and get features out of URL, if extra characters - string matching 
#List of authentic sources - stirng matching if complete match - good score, close match - bad score, distanc is far - neutral

In [17]:
#Testing Data
#Headline Column
test_headLine = x_test.loc[:,'Headline']

test_docHeadLineWordArray = []
for i in range(x_test.shape[0]):
    test_docHeadLineWordArray.append(test_headLine.iloc[i])
        
test_HeadLineArray = list(map(lambda x: tokenizeDocument(x), test_docHeadLineWordArray))
print("Preprocessing Completed for HeadLine of testing data")

Preprocessing Completed for HeadLine of testing data


In [18]:
#Testing Data
#Body Column
test_body = x_test.loc[:,'Body']

test_docBodyWordArray = []
for i in range(x_test.shape[0]):
    test_docBodyWordArray.append(test_body.iloc[i])
        
test_BodyArray = list(map(lambda x: tokenizeDocument(x), test_docBodyWordArray))
print("Preprocessing Completed for Body of testing data")

Preprocessing Completed for Body of testing data


In [19]:
train_headLineVector = []
#Headline Word Vectors
for i in range(x_train.shape[0]):
    train_headLineVector.append(model.infer_vector(train_HeadLineArray[i]))

test_headLineVector = []
#Headline Word Vectors
for i in range(x_test.shape[0]):
    test_headLineVector.append(model.infer_vector(test_HeadLineArray[i]))    
    
test_bodyVector = []
#Headline Word Vectors
for i in range(x_test.shape[0]):
    test_bodyVector.append(model.infer_vector(test_BodyArray[i]))

In [28]:
import numpy

np_trainHeadline = numpy.array([numpy.array(xi) for xi in train_headLineVector]) 

train_bodyVector = []
for i in range(x_train.shape[0]):
    train_bodyVector.append(model.docvecs[i])
np_trainBody = numpy.array([numpy.array(xi) for xi in train_bodyVector])

inp_x_train = []
for i in range(x_train.shape[0]):
    inp_x_train.append(numpy.concatenate((np_trainHeadline[i],np_trainBody[i])))

inp_x_train = numpy.array(inp_x_train)

In [29]:
inp_x_train.shape

(3190, 600)

In [30]:
np_testHeadline = numpy.array([numpy.array(xi) for xi in test_headLineVector])
np_testBody = numpy.array([numpy.array(xi) for xi in test_bodyVector])

inp_x_test = []
for i in range(x_test.shape[0]):
    inp_x_test.append(numpy.concatenate((np_testHeadline[i],np_testBody[i])))

inp_x_test = numpy.array(inp_x_test)

In [31]:
inp_x_test.shape

(798, 600)

In [33]:
from sklearn import svm

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

param_grid = [
    #{'C': C, 'kernel': ['linear']},
    #{'C': C, 'gamma': [0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']},
    {'degree': [2,3,4], 'kernel': ['poly']},
    #{'coef0': [0.0], 'kernel': ['sigmoid']} 
]

table = {}

score_metric = 'accuracy'
clf = GridSearchCV(svm.SVC(), param_grid, cv = 5, scoring = score_metric)
clf.fit(inp_x_train, y_train)
print("Best parameters set found :", clf.best_params_)
means = clf.cv_results_['mean_test_score']

for mean, params in zip(means, clf.cv_results_['params']):
    if params == clf.best_params_:
        print("%s -> %0.3f" % (score_metric, mean))
    key = str(params)
    if key not in table:
        table[key] = []
    table[key].append("%0.3f" % (mean))
print()



Best parameters set found : {'degree': 2, 'kernel': 'poly'}
accuracy -> 0.899



In [35]:
from sklearn.metrics import accuracy_score

svm_model = svm.SVC(degree = 2, kernel = 'poly')
svm_model.fit(inp_x_train, y_train)
y_pred = svm_model.predict(inp_x_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.543859649122807

In [ ]:
#stopwords.words('english')

In [ ]:
#punctuations = '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'
#punctuations

In [ ]:
#df.loc[1318,'Body']

In [ ]:
#df.loc[1318,'Label']

In [ ]:
#Intersect_word2vec_format
#max_epochs = 100
#vec_size = 300
#alpha = 0.025

#model1 = Doc2Vec(vector_size=vec_size,
#                alpha=alpha, 
#                min_alpha=0.025,
#                min_count=5,
#                window = 10,
#                dm =1)

#model1.build_vocab(tagged_data)

#model1.intersect_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  # C binary format

In [ ]:
import numpy
numpy.__version__